# T1531 - Account Access Removal
Adversaries may interrupt availability of system and network resources by inhibiting access to accounts utilized by legitimate users. Accounts may be deleted, locked, or manipulated (ex: changed credentials) to remove access to accounts. Adversaries may also subsequently log off and/or perform a [System Shutdown/Reboot](https://attack.mitre.org/techniques/T1529) to set malicious changes into place.(Citation: CarbonBlack LockerGoga 2019)(Citation: Unit42 LockerGoga 2019)

In Windows, [Net](https://attack.mitre.org/software/S0039) utility, <code>Set-LocalUser</code> and <code>Set-ADAccountPassword</code> [PowerShell](https://attack.mitre.org/techniques/T1059/001) cmdlets may be used by adversaries to modify user accounts. In Linux, the <code>passwd</code> utility may be used to change passwords. Accounts could also be disabled by Group Policy. 

Adversaries who use ransomware may first perform this and other Impact behaviors, such as [Data Destruction](https://attack.mitre.org/techniques/T1485) and [Defacement](https://attack.mitre.org/techniques/T1491), before completing the [Data Encrypted for Impact](https://attack.mitre.org/techniques/T1486) objective. 

## Atomic Tests

### Atomic Test #1 - Change User Password - Windows
Changes the user password to hinder access attempts. Seen in use by LockerGoga. Upon execution, log into the user account "AtomicAdministrator" with
the password "HuHuHUHoHo283283".

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
net user AtomicAdministrator User2ChangePW! /add
net.exe user AtomicAdministrator HuHuHUHoHo283283@dJD
```

In [ ]:
Invoke-AtomicTest T1531 -TestNumbers 1

### Atomic Test #2 - Delete User - Windows
Deletes a user account to prevent access. Upon execution, run the command "net user" to verify that the new "AtomicUser" account was deleted.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
net user AtomicUser User2DeletePW! /add
net.exe user AtomicUser /delete
```

In [ ]:
Invoke-AtomicTest T1531 -TestNumbers 2

### Atomic Test #3 - Remove Account From Domain Admin Group
This test will remove an account from the domain admins group

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Requires the Active Directory module for powershell to be installed.

##### Check Prereq Commands:
```powershell
if(Get-Module -ListAvailable -Name ActiveDirectory) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Add-WindowsCapability -Online -Name "Rsat.ActiveDirectory.DS-LDS.Tools~~~~0.0.1.0"

```

In [ ]:
Invoke-AtomicTest T1531 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$PWord = ConvertTo-SecureString -String password -AsPlainText -Force
$Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList domain\super_user, $PWord
if((Get-ADUser remove_user -Properties memberof).memberof -like "CN=Domain Admins*"){
  Remove-ADGroupMember -Identity "Domain Admins" -Members remove_user -Credential $Credential -Confirm:$False
} else{
    write-host "Error - Make sure remove_user is in the domain admins group" -foregroundcolor Red
}
```

In [ ]:
Invoke-AtomicTest T1531 -TestNumbers 3

## Detection
Use process monitoring to monitor the execution and command line parameters of binaries involved in deleting accounts or changing passwords, such as use of [Net](https://attack.mitre.org/software/S0039). Windows event logs may also designate activity associated with an adversary's attempt to remove access to an account:

* Event ID 4723 - An attempt was made to change an account's password
* Event ID 4724 - An attempt was made to reset an account's password
* Event ID 4726 - A user account was deleted
* Event ID 4740 - A user account was locked out

Alerting on [Net](https://attack.mitre.org/software/S0039) and these Event IDs may generate a high degree of false positives, so compare against baseline knowledge for how systems are typically used and correlate modification events with other indications of malicious activity where possible.

## Shield Active Defense
### System Activity Monitoring 
 Collect system activity logs which can reveal adversary activity. 

 Capturing system logs can show logins, user and system events, etc.  Collecting this data and potentially sending it to a centralized location can help reveal the presence of an adversary and the actions they perform on a compromised system.
#### Opportunity
There is an opportunity to create a detection with a moderately high probability of success.
#### Use Case
A defender can implement monitoring to alert if a user account is altered outside normal business hours, from remote locations, etc.
#### Procedures
Ensure that systems capture and retain common system level activity artifacts that might be produced.
Monitor Windows systems for event codes that reflect an adversary changing passwords, adding accounts to groups, etc.